# FIT5201: Assessment 1
## The Elements of Machine Learning

### Objectives
This assignment consists of three parts (A,B,C) that assess your understanding of model complexity, model selection, uncertainty in prediction with bootstrapping, and probabilistic machine learning. The total marks of this assessment is 100, and will contribute to the 20% of your final score. 

### Part A.  Model Complexity and Model Selection
In this part, you study the effect of model complexity on the training and testing errors.  You also demonstrate your programming skills by developing a regression algorithm and a cross-validation technique that will be used to select the models with the most effective complexity.

__Background__. A KNN regressor is similar to a KNN classifier (covered in Activity 1.1) in that it finds the K nearest neighbors and estimates the value of the given test point based on the values of its neighbours. The main difference between KNN regression and KNN classification is that KNN classifier returns the label that has the majority vote in the neighborhood, whilst KNN regressor returns the average of the neighbors’ values. 


##### Q1-1 KNN regressor function

Q1-1) Implement the KNN regressor function:
                                     knn(train.data, train.label, test.data, K=3) 
which takes the training data and their labels (continuous values), the test set, and the size of the neighborhood (K). It should return the regressed values for the test data points. When choosing the neighbors, you can use the Euclidean distance function to measure the distance between a pair of data points. 

__Hint__: You are allowed to use KNN classifier code from Activity 1 of Module 1.

**Answer**

In [ ]:
library(reshape2)
library(ggplot2)
library(corrplot)

In [ ]:
# KNN function (distance should be one of euclidean, maximum, manhattan, canberra, binary or minkowski)
knn <- function(train.data, train.value, test.data, K=3, distance = 'euclidean'){
        ## count number of test samples
        train.len <- nrow(train.data)
        ## count number of test samples
        test.len <- nrow(test.data)

        ## calculate distances between samples
        dist <- as.matrix(dist(rbind(test.data, train.data), method= distance))[1:test.len, (test.len+1):(test.len+train.len)]

        test.predict <- vector(mode="numeric", length=test.len)
            ## for each test sample...
        for (i in 1:test.len){
                ### ...find its K nearest neighbours from training sampels...
            nn <- as.data.frame(sort(dist[i,], index.return = TRUE))[1:K,2]
                ## nn is the index of training data and also the column index of distance matrix

                ###... and calculate the predicted labels according to the majority vote
            test.predict[i]<- (mean(train.value[nn]))
            }
    ## return the class labels as output
    return (round(test.predict,2))
}


Q1-2) Plot the training and the testing errors versus 1/K for K=1,..,20 in one plot, using the Task1A_train.csv and Task1A_test.csv datasets provided for this assignment. 

**Answer**

In [ ]:
# load the training data
train.data <- read.csv('./assessments_datasets/Task1A_train.csv')
train.value <- train.data[,2]
train.data <- train.data[,-2]
train.data <- as.data.frame(train.data) # convert train.data from vector to dataframe
names(train.data) <- 'X'

In [ ]:
# load the testing data
test.data <- read.csv('./assessments_datasets/Task1A_test.csv')
test.value <- test.data[,2]
test.data <- test.data[,-2]
test.data <- as.data.frame(test.data)  # convert test.data from vector to dataframe
names(test.data) <- 'X'

In [ ]:
# create a dataframe with k rows to store the training and testing errors for each k
MSE1 = data.frame('K'= 1:20, 'train' = rep(0,20), 'test' = rep(0,20))
# populate the table with training error and testing error
# the error is calculated by mean squared error
for ( k in 1:20) {
    MSE1[k, 'test'] = mean((knn(train.data,train.value,test.data,K=k)-test.value)^2)
    MSE1[k, 'train'] = mean((knn(train.data,train.value,train.data,K=k)-train.value)^2)
}
MSE.m <- melt(MSE1, id = 'K') 

In [ ]:
names(MSE.m) <- c('K','type','error')
ggplot(data=MSE.m, aes(x=1/K, y=error, color=type)) + geom_line() +
       scale_color_discrete(guide = guide_legend(title = NULL)) + theme_minimal() +
       ggtitle("Mean Squared Error for training set and testing set")

Q1-3 Report the best value for K in terms of the testing error. Discuss the values of K corresponding to underfitting and overfitting based on your plot in the previous sub-question, Q1-2. 

**Answer**

The best value of K should be the one who gives the lowest test error. According to the error plot above, we found that the optimal 1/k lies somewhere near 0.1, with the help of the test error table below, we can identify the optimal k which yields the lowest testing error is 11. 

From the graph above we can observe two kinds of trend of testing error and training error, the training MSE declines monotonically as 1/k increases whilst the test MSE initially declines before it reaches the optimal point, then it starts to increase again because the model overfits the training model and raise the error caused by variance. Since overfitting happens when the model fits the training set too closely to let randomness and outliers affect the model so that it will perform poorly on testing set. This characteristic can be found in the plot when k ranges from 1 to 12, which shows the training error keeps declining while testing error experience an increase, so it is fair to say from 12 onwards, the model is overfitting. Underfitting happens when the model is simpler than the true model and it fails to capture real trend of data points for both training and testing set. As can be seen in the graph, the MSE of both training set and testing set are declining when 1/k is from 1/20 t0 1/13, this is overfitting as larger k is, the simpler model will be.

#### Question 2 [K-fold Cross Validation, 20 Marks]  

Q2-1) Implement a K-fold Cross Validation (CV) function for your KNN regressor:  
       cv(train.data, train.label, numFold=10) 
which takes the training data and their labels (continuous values), the number of folds, and returns errors for different folds of the training data. 

__Hint__: you are allowed to use bootstrap code from Activity 2 of Module 1.

**Answer**

In [ ]:
# this function will give the random index of testing data in each fold
getTest <- function (fold = 10, train.length = 42){
    indices <- 1: train.length  # get the index of training data
    indices <- sample(indices, length(indices))  # shuffle the index
    indices.partA <- indices[1:(length(indices)- (length(indices) %% fold))]  # split the index into each folds
    size <- length(indices.partA)/fold
    test.indices <- split(indices.partA, cumsum( (1:length(indices.partA)-1)%% size ==0) )
    if  (length(indices) %% fold != 0 ) {  # assign the remainder index to other folds
        for ( i in 1: (length(indices) %% fold)  ) {
            test.indices[[i]] <- c(test.indices[[i]], indices[length(indices.partA)+i])
        }
    }
    return (test.indices) # return a list, each element is the index of testing data for a fold
}

In [ ]:
# this function will return the matrix of testing error, each row represents the number of nearest neighbour
# each column represents the error in each fold
cv <- function (train.data, train.value, numFold = 10, knear = 20) {
   test.index <- getTest( fold = numFold, train.length = nrow(train.data) )
   cvError <- matrix(nrow = knear, ncol = numFold) 
   for ( k in 1: knear){
       for ( f in 1:numFold ){
           test.data.chunk <- train.data[test.index[[f]],]  # get the test data in each fold
           test.data.chunk <- as.data.frame(test.data.chunk)
           test.value.chunk <- train.value[test.index[[f]]] # get the test value in each fold
           train.data.chunk <- train.data[-test.index[[f]],] # get the train data in each fold
           train.data.chunk <- as.data.frame(train.data.chunk)
           train.value.chunk <- train.value[-test.index[[f]]] # get the train data in each fold
           
           names(train.data.chunk) <- 'X'
           names(test.data.chunk) <- 'X'
           cvError[k,f] <- mean((knn(train.data.chunk, train.value.chunk, test.data.chunk, K = k) - test.value.chunk)^2)
       }
   }
   return (cvError)
}

Q2-2) Using the training data, run your K-fold CV where the numFold is set to 10. Change the value of K=1,..,20 and for each K compute the average 10 error numbers you have got.  Plot the average error numbers versus 1/K for K=1,..,20. Further, add two dashed lines around the average error indicating the average +/- standard deviation of errors. Include the plot in your report.

**Answer**

In [ ]:
# aggregate the matrix to generate the mean and standard deviaion of error
kfold.result <- matrix(nrow = 20, ncol = 4)
cvError <- cv(train.data, train.value)
for (k in 1:20){
    kfold.result[k,1] <- k
    kfold.result[k,2] <- mean(cvError[k,])
    kfold.result[k,3] <- mean(cvError[k,]) + sd(cvError[k,])
    kfold.result[k,4] <-  mean(cvError[k,]) - sd(cvError[k,])
}
kfold.result <- as.data.frame(kfold.result)
names(kfold.result) <- c("k","MSE","plus_sd","minus_sd")

In [ ]:
# melt the matrix 
kfold.result.m <- melt(kfold.result, id = "k")

In [ ]:
# plot the MSE and their range within 1 standard deviation, color and line type are used to differentiate the lines
ggplot(data=kfold.result.m, aes(x=1/k, y= value, color=variable)) + geom_line(aes(linetype=variable, color=variable)) +
       scale_color_discrete(guide = guide_legend(title = NULL)) + theme_minimal() +
       ggtitle("K-fold cross validation") 

Q2-3) Report the values of K that result to minimum average error and minimum standard deviation of errors based on your cross validation plot in the previous sub-question, Q2-2.

**Answer**
Based on the graph above and the table below, we can see that when 1/k equals 0.5, i.e. k is 2, the mean squared error reached the lowest point and the distance between 1 sd above and below is most narrow, which indicates that 2 is the optimal value for k that can gives the lowest error and standard deviation.

### Part B. Prediction Uncertainty with Bootstrapping
This part is the adaptation of Activity 2 from KNN classification to KNN regression. You use the bootstrapping technique to quantify the uncertainty of predictions for the KNN regressor that you implemented in Part A. 

#### Question 3 [Bootstrapping, 20 Marks]
Q3-1) Modify the code in Activity 2 to handle bootstrapping for KNN regression. 

**Answer**

In [ ]:
# return the index for bootstrapping training set 
boot <- function (original.size=930, sample.size=25, times=100){
    indx <- matrix(nrow=times, ncol=sample.size) # create an empty matrix
    for (t in 1:times){
        indx[t, ] <- sample(x=original.size, size=sample.size, replace = TRUE)
    }
    return(indx)
}


Q3-2)Load Task1B_train.csv and Task1B_test.csv sets. Apply your bootstrapping for KNN regression with times = 100 (the number of subsets), size = 25 (the size of each subset), and change K=1,..,20 (the neighbourhood size). Now create a boxplot where the x-axis is K, and the y-axis is the average error (and the uncertainty around it) corresponding to each K.  
**Answer**

In [ ]:
# load the training and testing data
trainB <- read.csv('./assessments_datasets/Task1B_train.csv')
testB <- read.csv('./assessments_datasets/Task1B_test.csv')
train.dataB <- trainB[,-5] # extract independent values of training data
train.valueB <- trainB[,5] # extract dependent values of training data
test.dataB <- testB[,-5] # extract independent values of testing data
test.valueB <- testB[,5] # extract dependent values of testing data

In [ ]:
boot.indx <- boot(nrow(train.dataB),25,100 )  # get the training index

In [ ]:
K = 20
L = 100

# a dataframe to track the number of missclassified samples in each case
MSE <- data.frame('K'=1:K, 'L'=1:L, 'test'=rep(0,L*K))

# THIS MAY TAKE A FEW MINUTES TO COMPLETE
## for every k values:
for (k in 1: K){
    
    ### for every dataset sizes:
    for (l in 1:L){
        
        #### calculate iteration index i
        i <- (k-1)*L+l
        
        #### save sample indices that were selected by bootstrap
        indx <- boot.indx[l,]
        
        #### save the value of k and l
        MSE[i,'K'] <- k
        MSE[i,'L'] <- l
        
        #### calculate and record the train and test missclassification rates
        MSE[i,'test'] <-  mean((knn(train.dataB[indx,], train.valueB[indx], test.dataB, k)-test.valueB)^2)
    } 
}

In [ ]:
#  plot the boostrapping result for different k
ggplot(data=MSE, aes(factor(K), test, fill = 'red') )+ geom_boxplot(outlier.shape = NA)  + 
    scale_color_discrete(guide = guide_legend(title = NULL)) + 
    ggtitle('MSE vs. K (Box Plot)') + theme_minimal()


Q3-3) Based on your plot in the previous sub-question, Q3-2, how does the test error and its uncertainty behave as K increases? 

**Answer**

As k increases, the test error becomes larger while the uncertainty initially increases significantly and then become stable at a certain level.

Q3-4) Load Task1B_train.csv and Task1B_test.csv sets. Apply your bootstrapping for KNN regression with K=10 (the neighbourhood size), size = 25 (the size of each subset), and change times = 10, 20, 30,.., 200 (the number of subsets). Now create a boxplot where the x-axis is ‘times’, and the y-axis is the average error (and the uncertainty around it) corresponding to each value of ‘times’. 


In [ ]:
times <- seq(10,200,10) # get a vector of times
cumTimes <- cumsum(times)  # get a vector of cumulative value of times
total.size <- nrow(train.dataB)
btError <- matrix(nrow = sum(times), ncol = 3) # get a matrix to store value of MSE for each time 
btError <- as.data.frame(btError)
names(btError) <- c('T','indx','MSE')
for (t in times) {
    indx <- match(t, times) # get the position of t in times vector
    boot.index <- boot(total.size, 25, t)
    btError[(cumTimes[indx]-t+1):cumTimes[indx],'T'] <- t  # populate the first column with the value of times 
    btError[(cumTimes[indx]-t+1):cumTimes[indx],'indx'] <- 1:t  # populate the second column with the value of subset index
    for (i in 1:t) {
        indxA <- boot.index[i,]
        # populate the third column with the MSE of the subset 
        btError[cumTimes[indx]-t+i,'MSE']  <- (mean((knn(train.dataB[indxA,], train.valueB[indxA], test.dataB, K = 10)-test.valueB)^2))
    }
}

In [ ]:
#  plot the boostrapping result for different times
ggplot(data=btError, aes(factor(T), MSE, fill = 'red') )+ geom_boxplot(outlier.shape = NA)  + 
    scale_color_discrete(guide = guide_legend(title = NULL)) + 
    ggtitle('MSE vs. K (Box Plot)') + theme_minimal()


Q3-5) Based on your plot in the previous sub-question, Q3-4, how does the test error and its uncertainty behave as the number of subsets in bootstrapping increases? 

**Answer**

As the number of dataset increases, the errors tend to be stable and not deviate too much from each other.

### Part C. Probabilistic Machine Learning
In this part, you show your knowledge about the foundation of the probabilistic machine learning (i.e. probabilistic inference and modeling) by solving two simple but basic statistical inference problems. Solve the following problems based on the probability concepts you have learned in Module 1 with the same math conventions. Please show your work in your report. Also, there are two conceptual questions.


#### Question 4 [Bayes Rule, 20 Marks] 
Recall the simple example from Appendix A of Module 1. Suppose we have one red and one blue box. In the red box we have 2 apples and 6 oranges, whilst in the blue box we have 3 apples and 1 orange. Now suppose we randomly selected one of the boxes and picked a fruit. If the picked fruit is an apple, what is the probability that it was picked from the blue box?

Note that the chance of picking the red box is 40% and the selection chance for any of the pieces from a box is equal for all the pieces in that box.

**Answer**
<img src="Q4.png">


#### Question 5 [Maximum Likelihood, 20 Marks] 
As opposed to a coin which has two faces, a dice has 6 faces. Suppose we are given a dataset which contains the outcomes of 10 independent tosses of a dice: D:={1,4,5,3,1,2,6,5,6,6}. We are asked to build a model for this dice, i.e. a model which tells what is the probability of each face of the dice if we toss it. Using the maximum likelihood principle, please determine the best value for our model parameters.

Hint: You can use a multinomial distribution with 6 probability parameters, each of which corresponding to a dice face (as opposed to coin where there are two parameters). You need to form the likelihood objective function, and then maximise it by setting the derivative with respect to the parameters to zero. Since the probabilities must sum up to 1, you only need to maximise the likelihood objective with respect to five parameters; the value of the sixth parameter is then going to be one minus the sum of the other parameters.

**Answer**
<img src = "Q5-1.png">
<img src = "Q5-2.png">

#### Question 6: 
As you have seen through the module, you are generally in the position of choosing a less flexible or more flexible model for your regression or classification problem. You need to be aware that we choose a model to serve to our final goal regardless of the flexibility level. It means that we may prefer a less flexible model to a more flexible model, or vice versa. What are the advantages and disadvantages of a very flexible (versus a less flexible) approach for regression or classification? Under what circumstances might a more flexible approach be preferred to a less flexible approach? When might a less flexible approach be preferred?

**Answer**

The advantages of flexibale model is that it will fit well on the trainning data and yields a very low training error. As the level of flexibility increases, the model will fit the observed training data more closely. Because the flexible will capture the slight variations in the data, it will result in very low bias. The disadvantage of flexible models is that they tend to cause high variance of test error because changing any
one of the data points may cause estimates to change considerably. The flexible model works too hard to find patterns in the training data and
may be picking up some patterns that are just caused by random chance
rather than by true properties of the unknown true model, which is known as over-fitting.


when the true decision boundary is complex and the true model of data has high level of freedom, a flexible method is preferred. If we come from a variance-bias point of view, when the error is dominated by bias rather than variance, a more flexible model will contribute more to the decrease of error. However when variance has more impact on the error than bias, i.e. as the flexibility rise up, the variance will increase rapidly, in this case a more restricted model will outperform the high flexible model.
#### Question 7: 
Describe the differences between a parametric and a non-parametric statistical learning approach. What are the advantages of a parametric approach to regression or classification (as opposed to a non- parametric approach)? What are its disadvantages?

**Answer**

A parametric approach reduces the problem of estimating f down to one of estimating a set of parameters because it assumes a form for f.

A non-parametric approach does not assume a patricular form of f and so requires a very large sample to accurately estimate f.

The advantages of a parametric approach to regression or classification are the simplifying of modeling ff to a few parameters and not as many observations are required compared to a non-parametric approach.

The disadvantages of a parametric approach to regression or classification are a potentially inaccurate estimate f if the form of f assumed is wrong or to overfit the observations if more flexible models are used.